# *CubeGLM-only notebook*: Template to analyze a new batch of images, whole plate statistics only
**Notebook template for applying routine hyperspectral/segmentation cross-analysis phenomics workflow over new datasets without segmentation**

## Instructions for use
1.  Enter information for the experiment below
2. Set <font color=blue>variables</font> for data paths and parameters, as instructed by colored boxes.
3. "Save as" with filename describing experiment and anything special about this analysis (e.g. T18_OD_TAO_wk7_automation_test_attempt2.ipynb)
4. Run notebook from console, by either one of two methods:
- In the top bar of JupyterLab, select "Run" and "Run all cells"
- In a terminal, enter the below command with the notebook filename inserted<br>
```jupyter nbconvert --to HTML --ExecutePreprocessor.timeout=-1 --allow-errors --execute insert_filename_here```
5. Wait for email

## Experiment ID and quick description:

<div class="alert alert-block alert-success">
Provide a short description of the experiment in the below box. This should include unique identifier codes for the experiment, along with a short description of genotypes and treatments studied. The timepoint should also be included. </div>

## Parameters for analysis:

<div class="alert alert-block alert-success">
The below variables must be modified appropriately every time this workflow is run over new images.
</div>

### Data location
The `data` variable below provides the **complete** path to the folder containing data to be analyzed. This should include all folders and subfolders in which the data of interest is organized by. For the organizational system used for our lab's data, this should follow the format "/Experiment/Subexperiment/Timepoint/"

In [1]:
data_prefix="/media/michael/Elements_13/"

In [2]:
data_suffix="/Nathan_GRF/Nathan_GRF_12_2021/717_1xCWT_GRF/week_7/"

In [3]:
data="${data_prefix}${data_suffix}"

In [4]:
echo $data

/media/michael/Elements_13//Nathan_GRF/Nathan_GRF_12_2021/717_1xCWT_GRF/week_7/


### Sample information
Every experiment has a randomization datasheet, which was used to organize treatment and genotype information for each plate, prepare labels, and randomize plates. The path to this file is provided through the `randomization_datasheets` variable below. This workflow requires this datasheet in order to know which plates have which genotype/treatment. At a later date, we will integrate an ability to read this data directly from labels.

In [5]:
randomization_datasheet="${data_prefix}/Nathan_GRF/Nathan_GRF_12_2021/717_1xCWT_GRF/1xCWT_wk3.xlsx"

In [6]:
ls $randomization_datasheet

/media/michael/Elements_13//Nathan_GRF/Nathan_GRF_12_2021/717_1xCWT_GRF/1xCWT_wk3.xlsx


### Detection of missing or contaminated explants

Set the `missing_explants` variable to `"Automatic"` if using model to automatically detect missing and contaminated explants. Note that this model is only supported for plates with 12 explants. Otherwise, set this option to `"Manual"` and include this information in the `total_explants_manual` column in the randomization datasheet.

In [7]:
missing_explants="Automatic"

Enter your email where results will be sent

### Email

In [8]:
email=michael.nagle@oregonstate.edu
















<div class="alert alert-block alert-warning">
All variables below should be modified only as needed to indicate the fluorescent proteins in samples and the grid layout of explants. </div>

### Fluorescent protein settings

Below should be all known fluorescent components contained in the sample. This includes each fluorescent protein, as well as a "noise" or "diffraction" term if applicable. All of these components must exist in the user's spectral library. `GMOdetector` currently comes with a spectral library that includes, by default:
- DsRed
- ZsYellow
- GFP
- Chl
- ChlA
- ChlB
- Diffraction

In [9]:
fluorophores=(DsRed Chl Diffraction) # An explanation of array variables in bash is here: https://tldp.org/LDP/Bash-Beginners-Guide/html/sect_10_02.html

The user has the option of limiting loading of hyperspectral data and subsequent regression to a specific range of wavelengths, using the `desired_wavelength_range` array variable. This range should cover all fluorophores provided in the `fluorophores` array variable able. Aside from runtime, there is no disadvantage to including a wider range than is needed.

In [10]:
desired_wavelength_range=(500 900)

### False color plot settings

To assist user inspection of regression results, false color plots can be produced by `GMOdetector` to show results of regression over whole samples. 
Note: In v0.2.0.x of the workflow, these parameters are independent of those used later by `GMOlabeler` to produce by-explant plots including false color plots. These will be made the same in a later update.
The `FalseColor_channels` array variable indicates the components to be plotted as red, green and blue, in that order.

In [11]:
FalseColor_channels=(Chl DsRed Diffraction)

The `FalseColor_caps` array variable indicates an upper limit of signal for each of these component. Any signal at or above these values will appear with maximum brightness; thus, these variables are comparable to exposure on a camera. If caps are too high, not much signal at lower ranges will be seen. If cap for a given component is too low, the false color images will appear overexposed with respect to the component.

In [12]:
FalseColor_caps=(200 400 200)

`GMOlabeler` will use the below parameters to classify individual explants on plates as positive for a given fluorescent protein or not. A single fluorescent protein is chosen for the by-explant false color plots. Multiple can be used to generate the final plots summarizing results.

In [13]:
reporters=(DsRed)
#reporter=DsRed

Parameters for reporter signal threshold and pixel threshold must be provided by the user. Our grid search yielded several noted below. These were most recently calculated from statistics produced with Python `GMOdetector`.

<img src="Figures/GMOlabeler_parameters.png">

In [14]:
reporter_threshold=38

### Grid settings

This is only needed if we're using a model to detect missing explants. Note that this model only works for the 12-explant grid.

In [15]:
pre_aligned_resized_grid_borders=("256,490,1720,1602")

<div class="alert alert-block alert-danger">
The below variables do not need to be modified during any routine use of the workflow.
</div>

Set dimensions for plot outputs

In [16]:
#width=15
width=9
height=5

In [17]:
parallel=0

### Paths to workflow modules

These only need to be modified if you are setting up a `GMOnotebook` template on a new computer.

In [18]:
cwd="/home/michael/GMOnotebook"

In [19]:
gmodetector_wd="/home/michael/gmodetector_py/"
spectral_library_path="${gmodetector_wd}spectral_library/"
deeplab_path="/home/michael/poplar_model_2_w_contam/"
alignment_path="/home/michael/ImageAlignment/"
gmolabeler_path="/home/michael/GMOlabeler/"
contamination_path="/home/michael/DenseNet"
data_prefix="/home/michael/data/"
output_directory_prefix="${data_prefix}gmodetector_out/"

<div class="alert alert-block alert-info">
With all above variables set, please "Save as..." with a filename referencing this specific dataset. <br>Finally, deploy the workflow (Step 4 in above instructions).
</div>

# Automated workflow to be deployed

See the below code for a walkthrough of how GMOnotebook works, or view the outputs after running the workflow for help troubleshooting errors in specific steps of analysis.

<div class="alert alert-block alert-danger"> <b>Danger:</b> Do not modify any below code without creating a new version of the template notebook. During routine usage, this workflow should be customized only by modifying variables above, while leaving the below code unmodified. </div>

These internal variables are set automatically.

In [20]:
datestamp=$(date +”%Y-%m-%d”)

In [21]:
data_folder=$(echo $data | cut -d/ -f5-)

In [22]:
timepoint="$(basename -- $data_folder)"

In [23]:
output_directory_full="$output_directory_prefix$data_folder"

In [24]:
#dataset_name=$(echo $data_folder | sed -e 's/\//-/g')
dataset_name=$(echo $data_folder | sed -e 's/\///g')

In [25]:
echo $dataset_name

Nathan_GRFNathan_GRF_12_2021717_1xCWT_GRFweek_7


Time analysis begins:

In [26]:
echo $(date)

Wed 26 Jan 2022 03:28:16 PM PST


## Quantification of fluorescent proteins by regression

The Python package `CubeGLM` is used to quantify fluorescent proteins in each pixel of hyperspectral images via linear regression. Hyperspectral images are regressed over spectra of known components, and pixelwise maps of test-statistics are constructed for each component in the sample. This approach to quantifying components of hyperspectral images is described in-depth in the Methods section from <a href="https://link.springer.com/article/10.1007/s40789-019-0252-7" target="_blank">Böhme, et al. 2019</a>. Code and documentation for `CubeGLM` is on <a href="https://github.com/naglemi/GMOdetector_py" target="_blank">Github</a>.

In [27]:
cd $gmodetector_wd

In [28]:
job_list_name="$dataset_name.jobs"

In [29]:
rm -rf $job_list_name

In [30]:
for file in $data/*.hdr
do
 if [[ "$file" != *'hroma'* ]] && [[ "$file" != *'roadband'* ]]; then
  echo "python -W ignore wrappers/analyze_sample.py \
--file_path $file \
--fluorophores ${fluorophores[*]} \
--min_desired_wavelength ${desired_wavelength_range[0]} \
--max_desired_wavelength ${desired_wavelength_range[1]} \
--red_channel ${FalseColor_channels[0]} \
--green_channel ${FalseColor_channels[1]} \
--blue_channel ${FalseColor_channels[2]} \
--red_cap ${FalseColor_caps[0]} \
--green_cap ${FalseColor_caps[1]} \
--blue_cap ${FalseColor_caps[2]} \
--plot 1 \
--spectral_library_path "$spectral_library_path" \
--output_dir $output_directory_full \
--threshold 38" >> $job_list_name
 fi
done

In [31]:
echo $job_list_name

Nathan_GRFNathan_GRF_12_2021717_1xCWT_GRFweek_7.jobs


In [32]:
conda activate test-environment

if [ $parallel -eq 1 ]
then
    parallel -a $job_list_name
fi

if [ $parallel -eq 0 ]
then
    bash $job_list_name
fi

conda deactivate
conda deactivate

(test-environment) (test-environment) (test-environment) (test-environment) Running GMOdetector version 0.0.875
load mode isload_full_then_crop
Saving to /home/michael/data/gmodetector_out//Nathan_GRF/Nathan_GRF_12_2021/717_1xCWT_GRF/week_7/CWT1_I5.0_F1.9_L100LED30_120305_0_0_0_Fluorescence_weights.hdf with key of weights
Saving summary stats w/ threshold38.0 to /home/michael/data/gmodetector_out//Nathan_GRF/Nathan_GRF_12_2021/717_1xCWT_GRF/week_7/CWT1_I5.0_F1.9_L100LED30_120305_0_0_0_Fluorescence_summary.csv
Producing image channel for DsRed with cap 400 in color green
Producing image channel for Chl with cap 200 in color red
Producing image channel for Diffraction with cap 200 in color blue
Saving image to /home/michael/data/gmodetector_out//Nathan_GRF/Nathan_GRF_12_2021/717_1xCWT_GRF/week_7/CWT1_I5.0_F1.9_L100LED30_120305_0_0_0_Fluorescence.png
Saving image metadata to /home/michael/data/gmodetector_out//Nathan_GRF/Nathan_GRF_12_2021/717_1xCWT_GRF/week_7/CWT1_I5.0_F1.9_L100LED30_120

Time regression completes:

In [33]:
echo $(date)

Wed 26 Jan 2022 03:32:47 PM PST


## Classification of contaminated/missing explants

Plates are cropped into sub-images for each explant and each is analyzed to determine if the explant position should be excluded from analysis due to being missing or contamination. Missing and contaminated explants are recognized using a trained Densenet model (<a href="https://github.com/Contamination-Classification/DenseNet" target="_blank">Huang, et al. 2018</a>). Our fork of the Densenet repository is available on <a href="https://arxiv.org/abs/1608.06993" target="_blank">GitHub</a>.

<img src="Figures/Densenet.png">
Figure: These are four examples of contaminated explants used in the training set for this pre-trained model

To check the grid cropping dimensions, we can run the following script. Note that these are the dimensions to crop the image to after resizing to 2000x2000 (from 4000x4000 in the case of the *macroPhor Array).

#### Prepare list of images

In [34]:
if [ $missing_explants = "Automatic" ]; then
    echo "Missing explants will be inferred."
    cd $data
    ls -d $PWD/* $data | grep -i "rgb.jpg" > rgb_list.txt
    sed -i '/hroma/d' rgb_list.txt
    img_list_path="${data}/rgb_list.txt"
else
    echo "Missing explants input manually by user, in randomization datasheet"
    echo $missing_explants
fi

Missing explants will be inferred.


If the mode for missing explant data is automatic, prepare input file for script to detect missing explants and run this script.

#### Check dimensions for grid cropping

The models to detect contamination and missing explants require a user input to define the pixel boundaries of the grid along which explants are placed. Note that currently, only the 12-explant grid is supported. To use other grids, contamination and missing explant data must be provided manually in a file formatted just like an output from this script.

When running `inference.py` to detect missing or contaminated explants, the user should provide dimensions for cropping down to the grid borders. Note, these dimensions apply after the image is rescaled to 2000x2000.
- Default dimensions, used before the imager began to take "off-center" images, are (310, 515, 1750, 1610).
- Dimensions for cropping for the off-center images are (260, 600, 1700, 1710). 
- New dimensions (275, 438, 1725, 1535) are for images taken after camera settings were re-centered.<br>

We can test dimensions using the `--debug` option for `inference.py` as in the below code block. Next, we will run the same script in the regular mode to detect missing and contaminated explants using settled-upon cropping dimensions.

Uncomment and run this code block only if you wish to troubleshoot cropping.

In [35]:
# if [ $missing_explants = "Automatic" ]; then
#     cd $contamination_path
#     conda activate densenet
#     python inference.py --img-list=$img_list_path --crop_dims "(275,438,1725,1535)" --debug
#     conda deactivate
# else
#     echo "Missing explants input manually by user, in file: "
#     echo $missing_explants
# fi

The outputs will be saved in this folder and can be evaluated to check how well the cropping worked.

In [36]:
# echo $contamination_path

#### Infer contaminated/missing explants

Dependencies include `keras-preprocessing`, `termcolor`,  `protobuf` and `absl-py`

In [37]:
date

Wed 26 Jan 2022 03:32:48 PM PST


In [38]:
if [ $missing_explants = "Automatic" ]; then
    cd $contamination_path
    conda activate DenseNet
    python -W ignore inference.py \
    --img-list=$img_list_path \
    --crop_dims $pre_aligned_resized_grid_borders \
    --output_file=output.csv >> $data/log_contam.txt
    mv -f output.csv "${data}/output.csv"
    conda deactivate
fi

Using TensorFlow backend.
Instructions for updating:
Colocations handled automatically by placer.
2022-01-26 15:32:52.478060: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2022-01-26 15:32:52.511473: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 3391990000 Hz
2022-01-26 15:32:52.512467: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x55a07063ae60 executing computations on platform Host. Devices:
2022-01-26 15:32:52.512495: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>


In [39]:
if [ $missing_explants = "Automatic" ]; then
    missing_explants_sheet="${data}/output.csv"
    echo "Missing explants inferred by model and written to file:"
    echo $missing_explants_sheet
else
    missing_explant_sheet="Not applicable"
    echo "Missing explants input manually by user, in randomization datasheet"
fi

Missing explants inferred by model and written to file:
/media/michael/Elements_13//Nathan_GRF/Nathan_GRF_12_2021/717_1xCWT_GRF/week_7//output.csv


In [40]:
date

Wed 26 Jan 2022 03:33:45 PM PST


## Plot results

In [162]:
pwd

pwd
/home/michael/GMOlabeler/plots/Nathan_GRF/Nathan_GRF_12_2021/717_1xCWT_GRF/week_7


: 1

In [163]:
ls

ls
 gridplot_args.rds
 Nathan_GRF-Nathan_GRF_12_2021-717_1xCWT_GRF-week_7-.zip
'Whole Plate'
 Whole_plate_stats.csv


: 1

In [164]:
echo $missing_explants

echo $missing_explants
Automatic


: 1

In [165]:
conda activate gmolabeler
cd $gmolabeler_path
Rscript $cwd/Extras/Whole_plate_plots.R \
-d "${data_folder}/" \
-r "$randomization_datasheet" \
-m $missing_explants \
-M $missing_explants_sheet \
--height $height \
--width $width

conda deactivate

conda activate gmolabeler
(gmolabeler) cd $gmolabeler_path
(gmolabeler) Rscript $cwd/Extras/Whole_plate_plots.R \
-d "${data_folder}/" \
-r "$randomization_datasheet" \
-m $missing_explants \
-M $missing_explants_sheet \
--height $height \
--width $width
[1] "Saving list of input arguments to : /home/michael/GMOlabeler/plots//Nathan_GRF/Nathan_GRF_12_2021/717_1xCWT_GRF/week_7////gridplot_args.rds"
[1] "Missinge explant option:"
[1] "Automatic"

[1] "Calculating total numbers of missing explants per plate"
[1] "Chl"
[1] 13
[1] "Diffraction"
[1] 13
[1] "DsRed"
[1] 13
[1] "Intercept"
[1] 13
[1] "Chl"
[1] "Diffraction"
[1] "DsRed"
[1] "Intercept"
(gmolabeler) 
(gmolabeler) conda deactivate


: 1

In [166]:
echo $randomization_datasheet

echo $randomization_datasheet
/media/michael/Elements_13//Nathan_GRF/Nathan_GRF_12_2021/717_1xCWT_GRF/1xCWT_wk3.xlsx


: 1

## Email plots to user

### ZIP results

In [167]:
echo "${gmolabeler_path}/plots${data_folder}"

echo "${gmolabeler_path}/plots${data_folder}"
/home/michael/GMOlabeler//plots/Nathan_GRF/Nathan_GRF_12_2021/717_1xCWT_GRF/week_7/


: 1

In [168]:
cd "${gmolabeler_path}/plots/${data_folder}"

cd "${gmolabeler_path}/plots/${data_folder}"


: 1

In [169]:
rm -f ./plants_over_plates.csv

rm -f ./plants_over_plates.csv


: 1

In [170]:
echo ${gmolabeler_path}/output/${data_folder}/

echo ${gmolabeler_path}/output/${data_folder}/
/home/michael/GMOlabeler//output//Nathan_GRF/Nathan_GRF_12_2021/717_1xCWT_GRF/week_7//


: 1

In [171]:
#cp "${gmolabeler_path}/output/${data_folder}/plants_over_plates.csv" ./

#cp "${gmolabeler_path}/output/${data_folder}/plants_over_plates.csv" ./


: 1

In [172]:
cp "${gmolabeler_path}/plots/${data_folder}/Whole Plate/Whole_plate_stats.csv" ./

cp "${gmolabeler_path}/plots/${data_folder}/Whole Plate/Whole_plate_stats.csv" ./


: 1

We don't use the version below because we just have one set of plots for all fluorescent components

In [173]:
#for reporter in ${reporters[@]}; do
#    cp "${gmolabeler_path}/output/${data_folder}/${reporter}/stats_with_sums_over_tissues.csv" ./
#done

#for reporter in ${reporters[@]}; do
#    cp "${gmolabeler_path}/output/${data_folder}/${reporter}/stats_with_sums_over_tissues.csv" ./
#done


: 1

In [174]:
#cd $data

#cd $data


: 1

In [175]:
rm -f Rplots.pdf

rm -f Rplots.pdf


: 1

In [176]:
cd ../

cd ../


: 1

This messy substitution is explained here: https://superuser.com/questions/1068031/replace-backslash-with-forward-slash-in-a-variable-in-bash

In [177]:
data_folder_Compress="${data_folder////-}.zip"
data_folder_Compress=${data_folder_Compress#?};

data_folder_Compress="${data_folder////-}.zip"
data_folder_Compress=${data_folder_Compress#?};


: 1

In [178]:
echo $data_folder

echo $data_folder
/Nathan_GRF/Nathan_GRF_12_2021/717_1xCWT_GRF/week_7/


: 1

In [179]:
echo $data_folder_Compress

echo $data_folder_Compress
Nathan_GRF-Nathan_GRF_12_2021-717_1xCWT_GRF-week_7-.zip


: 1

In [180]:
pwd

pwd
/home/michael/GMOlabeler/plots/Nathan_GRF/Nathan_GRF_12_2021/717_1xCWT_GRF


: 1

In [181]:
echo $timepoint

echo $timepoint
week_7


: 1

In [182]:
ls

ls
week_3  week_7


: 1

In [183]:
cd $timepoint
zip -r $data_folder_Compress ./*

cd $timepoint
zip -r $data_folder_Compress ./*
updating: gridplot_args.rds (stored 0%)
updating: Whole Plate/ (stored 0%)
updating: Whole Plate/Signif_Pixels_Diffraction.png (deflated 29%)
updating: Whole Plate/Signif_Pixels_Intercept.png (deflated 30%)
updating: Whole Plate/Total_Chl.png (deflated 32%)
updating: Whole Plate/Total_Diffraction.png (deflated 30%)
updating: Whole Plate/Whole_plate_stats.csv (deflated 70%)
updating: Whole Plate/Signif_Pixels_Chl.png (deflated 31%)
updating: Whole Plate/Total_Intercept.png (deflated 29%)
updating: Whole Plate/Total_DsRed.png (deflated 33%)
updating: Whole Plate/Signif_Pixels_DsRed.png (deflated 31%)
updating: Whole_plate_stats.csv (deflated 70%)


: 1

### Write email

In [184]:
duration=$(( SECONDS - start ))

duration=$(( SECONDS - start ))


: 1

https://unix.stackexchange.com/questions/53841/how-to-use-a-timer-in-bash

In [185]:
rm -f "${gmolabeler_path}/email_to_send.txt"
cp "${gmolabeler_path}/email_to_send_template.txt" "${gmolabeler_path}/email_to_send.txt"

rm -f "${gmolabeler_path}/email_to_send.txt"
cp "${gmolabeler_path}/email_to_send_template.txt" "${gmolabeler_path}/email_to_send.txt"


: 1

In [186]:
echo "" >> "${gmolabeler_path}/email_to_send.txt"
echo "Number of samples run: " >> "${gmolabeler_path}/email_to_send.txt"

echo "" >> "${gmolabeler_path}/email_to_send.txt"
echo "Number of samples run: " >> "${gmolabeler_path}/email_to_send.txt"


: 1

In [187]:
cat "${data}/test.csv" | wc -l >> "${gmolabeler_path}/email_to_send.txt"
echo "" >> "${gmolabeler_path}/email_to_send.txt"

cat "${data}/test.csv" | wc -l >> "${gmolabeler_path}/email_to_send.txt"
echo "" >> "${gmolabeler_path}/email_to_send.txt"


: 1

In [188]:
if (( $SECONDS > 3600 )) ; then
    let "hours=SECONDS/3600"
    let "minutes=(SECONDS%3600)/60"
    let "seconds=(SECONDS%3600)%60"
    echo "Completed in $hours hour(s), $minutes minute(s) and $seconds second(s)" >> "${gmolabeler_path}/email_to_send.txt"
elif (( $SECONDS > 60 )) ; then
    let "minutes=(SECONDS%3600)/60"
    let "seconds=(SECONDS%3600)%60"
    echo "Completed in $minutes minute(s) and $seconds second(s)" >> "${gmolabeler_path}/email_to_send.txt"
else
    echo "Completed in $SECONDS seconds" >> "${gmolabeler_path}/email_to_send.txt"
fi

if (( $SECONDS > 3600 )) ; then
    let "hours=SECONDS/3600"
    let "minutes=(SECONDS%3600)/60"
    let "seconds=(SECONDS%3600)%60"
    echo "Completed in $hours hour(s), $minutes minute(s) and $seconds second(s)" >> "${gmolabeler_path}/email_to_send.txt"
elif (( $SECONDS > 60 )) ; then
    let "minutes=(SECONDS%3600)/60"
    let "seconds=(SECONDS%3600)%60"
    echo "Completed in $minutes minute(s) and $seconds second(s)" >> "${gmolabeler_path}/email_to_send.txt"
else
    echo "Completed in $SECONDS seconds" >> "${gmolabeler_path}/email_to_send.txt"
fi


: 1

In [189]:
echo "" >> "${gmolabeler_path}/email_to_send.txt"

echo "" >> "${gmolabeler_path}/email_to_send.txt"


: 1

### Send email with results to user

In [190]:
pwd

pwd
/home/michael/GMOlabeler/plots/Nathan_GRF/Nathan_GRF_12_2021/717_1xCWT_GRF/week_7


: 1

In [191]:
echo $data_folder_Compress

echo $data_folder_Compress
Nathan_GRF-Nathan_GRF_12_2021-717_1xCWT_GRF-week_7-.zip


: 1

In [192]:
echo $data_folder

echo $data_folder
/Nathan_GRF/Nathan_GRF_12_2021/717_1xCWT_GRF/week_7/


: 1

In [193]:
echo $email

echo $email
michael.nagle@oregonstate.edu


: 1

In [194]:
echo "${gmolabeler_path}/email_to_send.txt"

echo "${gmolabeler_path}/email_to_send.txt"
/home/michael/GMOlabeler//email_to_send.txt


: 1

In [195]:
pwd

pwd
/home/michael/GMOlabeler/plots/Nathan_GRF/Nathan_GRF_12_2021/717_1xCWT_GRF/week_7


: 1

In [196]:
cat "${gmolabeler_path}/email_to_send.txt"

cat "${gmolabeler_path}/email_to_send.txt"

Your job is complete!

Number of samples run: 
13

Completed in 26 minute(s) and 18 second(s)



: 1

In [197]:
echo $data_folder_Compress

echo $data_folder_Compress
Nathan_GRF-Nathan_GRF_12_2021-717_1xCWT_GRF-week_7-.zip


: 1

In [198]:
du -sh $data_folder_Compress

du -sh $data_folder_Compress
396K	Nathan_GRF-Nathan_GRF_12_2021-717_1xCWT_GRF-week_7-.zip


: 1

In [199]:
cat "${gmolabeler_path}/email_to_send.txt"

cat "${gmolabeler_path}/email_to_send.txt"

Your job is complete!

Number of samples run: 
13

Completed in 26 minute(s) and 18 second(s)



: 1

Time workflow ends:

In [200]:
echo $(date)

echo $(date)
Wed 26 Jan 2022 03:54:39 PM PST


: 1

In [201]:
s-nail -a $data_folder_Compress -s $data_folder $email < "${gmolabeler_path}/email_to_send.txt"

s-nail -a $data_folder_Compress -s $data_folder $email < "${gmolabeler_path}/email_to_send.txt"
=

: 1